<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-30 09:30:44
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.32 C
-------------------
Today PnL: 4.14 K (0.03%)
Current PnL: -16.18 L (-11.28%)
CY Booked + Current PnL: -11.71 L (-8.16%)
-------------------
Total profit:  3.98 L
Total loss:  -20.16 L
-------------------
Total Booked + Current PnL: 15.27 L (13.1%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.38%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.34 L (60.0%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.41 C (106.37%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ADANIPORTS,1103.69,1583.0,2.51,M-LC,90.32,102929.0,24567.0,9469.0,0.77,31.35,9.20,43.43,71.0,2.59,0.80,37.67,XY24,NTT,MISC
49,LICI,785.05,983.0,-4.34,H-LC,88.17,234224.0,39532.0,9556.0,0.26,20.30,4.08,25.21,29.0,4.14,1.83,29.52,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.0,4.23,X-LC,25.81,154911.0,14102.0,11417.0,0.24,10.01,7.37,18.13,17.0,1.24,1.21,22.32,XY25,NTT,REFINERIES
56,PIDILITIND,2504.06,3576.0,-17.12,X-LC,24.73,91863.0,16741.0,15415.0,1.07,22.29,16.78,42.81,11.0,1.09,0.72,15.21,X40,BTT,CHEMICALS
55,PFIZER,4245.50,6318.3,11.68,H-SC,98.92,128975.0,31328.0,16341.0,0.40,32.08,12.67,48.82,138.0,1.92,1.01,43.28,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,6516.67,M-SC,97.85,72600.0,-21210.0,97828.0,9.39,-22.61,134.75,81.67,203.0,-0.22,0.57,40.59,XR,NTT,CERAMICS
72,SIS,477.00,528.00,2144.08,M-SC,53.23,60750.0,-20340.0,29008.0,2.88,-25.08,47.75,10.69,201.0,-0.70,0.47,23.12,X40N,NTT,MISC
83,TTKPRESTIG,769.29,1149.24,-9.35,X-SC,46.24,82864.0,-17913.0,67683.0,1.61,-17.77,81.68,49.39,65.0,-0.26,0.65,7.83,X40N,ATH,DURABLES
86,VAIBHAVGBL,350.21,521.00,65.84,X-SC,87.10,110739.0,-41252.0,115379.0,1.35,-27.14,104.19,48.77,59.0,-0.36,0.87,27.20,XR,NTT,JEWELLERY
18,CERA,8421.60,10420.45,-18.14,X-SC,93.55,84565.0,-24916.0,50900.0,1.32,-22.76,60.19,23.73,64.0,-0.49,0.66,27.99,X40N,ATH,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,866.51,M-SC,20.43,131421.0,-49722.0,95188.0,-3.75,-27.45,72.43,25.10,204.0,-0.52,1.03,5.19,XY24,NTT,HEALTHCARE
15,BSOFT,628.85,844.81,-24.48,M-SC,47.31,71094.0,-38955.0,76746.0,-3.45,-35.40,107.95,34.34,171.0,-0.51,0.56,17.04,XR,ATH,IT
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,19.35,62016.0,-51533.0,91585.0,-2.56,-45.38,147.68,35.27,238.0,-0.56,0.48,51.79,XR,NTT,HOTELS
75,SURYODAY,189.78,240.00,53.51,H-SC,59.14,109494.0,-40432.0,80106.0,-1.90,-26.97,73.16,26.46,129.0,-0.50,0.86,40.27,XR,NTT,BANKS
16,CAMPUS,294.86,393.00,NaN,NaN,81.72,159320.0,-8455.0,64302.0,-1.87,-5.04,40.36,33.28,175.0,-0.13,1.24,23.35,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.14,X-LC,51.61,247738.0,17139.0,113984.0,-0.35,7.43,46.01,56.86,1.0,0.15,1.94,14.66,X40,BTT,IT
80,TCS,3841.06,4998.00,-16.48,X-LC,23.66,240568.0,-24465.0,104286.0,-0.33,-9.23,43.35,30.12,2.0,-0.23,1.88,7.89,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.82,X-LC,72.04,183323.0,-16987.0,75786.0,-0.70,-8.48,41.34,29.35,4.0,-0.22,1.43,26.65,X200,ATH,IT
71,SIEMENS,4406.90,7969.85,-7.06,X-LC,86.02,84065.0,-26107.0,115177.0,0.40,-23.70,137.01,80.85,5.0,-0.23,0.66,23.81,AR,ATH,ELECTRICAL
32,HAVELLS,1588.02,2069.16,-6.32,X-LC,12.90,242299.0,-5432.0,80492.0,0.16,-2.19,33.22,30.30,6.0,-0.07,1.89,10.81,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,11.68,H-SC,98.92,128975.0,31328.0,16341.0,0.40,32.08,12.67,48.82,138.0,1.92,1.01,43.28,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.93,H-MC,61.29,102382.0,17281.0,25821.0,0.33,20.31,25.22,50.65,92.0,0.67,0.80,28.70,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,2.51,M-LC,90.32,102929.0,24567.0,9469.0,0.77,31.35,9.20,43.43,71.0,2.59,0.80,37.67,XY24,NTT,MISC
49,LICI,785.05,983.00,-4.34,H-LC,88.17,234224.0,39532.0,9556.0,0.26,20.30,4.08,25.21,29.0,4.14,1.83,29.52,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3576.00,-17.12,X-LC,24.73,91863.0,16741.0,15415.0,1.07,22.29,16.78,42.81,11.0,1.09,0.72,15.21,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,38.95,M-SC,64.52,144030.0,6452.0,87959.0,0.05,4.69,61.07,68.62,182.0,0.07,1.13,24.86,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,11.68,H-SC,98.92,128975.0,31328.0,16341.0,0.40,32.08,12.67,48.82,138.0,1.92,1.01,43.28,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.93,H-MC,61.29,102382.0,17281.0,25821.0,0.33,20.31,25.22,50.65,92.0,0.67,0.80,28.70,X40N,ATH,CABLES
28,GILLETTE,8109.44,10330.69,6.81,H-SC,83.87,157590.0,19730.0,18028.0,-0.49,14.31,11.44,27.39,118.0,1.09,1.23,31.85,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,2.51,M-LC,90.32,102929.0,24567.0,9469.0,0.77,31.35,9.20,43.43,71.0,2.59,0.80,37.67,XY24,NTT,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.22,H-SC,36.56,112095.0,10091.0,34929.0,0.77,9.89,31.16,44.14,111.0,0.29,0.88,27.74,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-30.66,M-MC,41.40,135194.0,6042.0,81022.0,-0.34,4.68,59.93,67.42,155.0,0.07,1.06,31.49,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,0.00,H-MC,78.49,152954.0,2041.0,58413.0,0.31,1.35,38.19,40.06,89.0,0.03,1.20,25.11,AR,ATH,AUTO
92,WIPRO,243.46,420.00,-10.16,M-LC,39.78,154256.0,3311.0,106144.0,-0.54,2.19,68.81,72.51,69.0,0.03,1.21,16.14,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,22.58,123262.0,-74.0,137043.0,-0.01,-0.06,111.18,111.05,108.0,-0.00,0.96,20.77,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-57.75,L-SC,19.35,62016.0,-51533.0,91585.0,-2.56,-45.38,147.68,35.27,238.0,-0.56,0.48,51.79,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-23.67,H-SC,56.99,86955.0,-11025.0,32895.0,-0.60,-11.25,37.83,22.32,130.0,-0.34,0.68,26.11,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,2.51,M-LC,90.32,102929.0,24567.0,9469.0,0.77,31.35,9.20,43.43,71.0,2.59,0.80,37.67,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,33.33,117342.0,-25137.0,44660.0,-0.21,-17.64,38.06,13.70,217.0,-0.56,0.92,38.50,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-380.10,H-SC,100.00,119158.0,-14128.0,98722.0,0.44,-10.60,82.85,63.47,131.0,-0.14,0.93,99.74,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-380.10,H-SC,100.00,119158.0,-14128.0,98722.0,0.44,-10.60,82.85,63.47,131.0,-0.14,0.93,99.74,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,11.68,H-SC,98.92,128975.0,31328.0,16341.0,0.40,32.08,12.67,48.82,138.0,1.92,1.01,43.28,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6516.67,M-SC,97.85,72600.0,-21210.0,97828.0,9.39,-22.61,134.75,81.67,203.0,-0.22,0.57,40.59,XR,NTT,CERAMICS
77,TANLA,917.30,1963.11,-34.99,H-SC,96.77,159745.0,-79670.0,352621.0,0.98,-33.28,220.74,114.01,110.0,-0.23,1.25,46.51,AR,ATH,IT
82,TRIDENT,37.35,48.00,64.93,M-SC,95.70,79287.0,-12968.0,39271.0,0.31,-14.06,49.53,28.51,220.0,-0.33,0.62,35.67,XR,NTT,TEXTILES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.94
1,25,41.15
2,50,70.07


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.02
XY25    14.42
X40     14.37
X40N    13.60
AR      12.49
XR      10.56
X200     1.43
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.47
M-SC    17.91
X-LC    17.48
H-MC     8.77
M-LC     7.79
H-LC     7.30
X-MC     5.64
M-MC     5.58
X-SC     2.84
L-SC     2.31
L-LC     1.17
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.46,-0.52,32.15
IT,9.97,-17.62,81.90
BANKS,9.74,-12.66,64.45
MISC,6.46,-30.55,88.30
FINANCE,5.92,-26.06,85.91
INSURANCE,5.44,4.09,24.66
PAINTS,5.41,-18.73,53.87
ELECTRICAL,5.09,-3.23,53.85
AUTO,4.49,-9.01,56.64


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2876564.0,26
AR,1249699.0,12
X40N,1211952.0,17
XR,1103703.0,14
X40,763402.0,11
XY25,455685.0,10
SR,196737.0,2
X200,75786.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2270303.0,21
M-SC,2029494.0,21
X-LC,782446.0,12
M-MC,553086.0,5
H-MC,505907.0,8
X-SC,346069.0,4
X-MC,312766.0,5
M-LC,308684.0,6
L-SC,301020.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      940142.0      6
M-SC       XY24      744641.0      6
H-SC       AR        549236.0      3
           X40N      499904.0      6
M-SC       X40N      473781.0      6
X-LC       X40       446840.0      5
M-MC       XY24      391060.0      3
M-SC       XR        287574.0      4
H-LC       AR        264025.0      3
H-SC       XR        246652.0      4
M-SC       X40       223799.0      2
M-LC       XY24      185344.0      4
H-MC       XY24      177639.0      2
L-SC       XR        154702.0      2
X-MC       XY24      144716.0      2
X-LC       AR        140332.0      2
M-SC       XY25      121549.0      1
X-LC       XY25      119488.0      4
X-SC       X40N      118583.0      2
           XR        115379.0      1
H-MC       XR        112230.0      1
X-SC       SR        112107.0      1
M-LC       XR        106144.0      1
X-MC       X40N       93863.0      2
M-SC       AR         93520.0      1
           SR         84630.0      1
L-SC       XY24       83149.0      1
M-MC       XR         81022.0      1
           AR         81004.0      1
X-LC       X200       75786.0      1
X-MC       XY25       74187.0      1
H-MC       XY25       73410.0      1
L-SC       AR         63169.0      1
H-MC       AR         58413.0      1
           X40        58394.0      2
L-LC       XY25       40299.0      1
H-SC       X40        34369.0      2
H-MC       X40N       25821.0      1
M-LC       XY25       17196.0      1
H-LC       XY25        9556.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
